# I. Import Module, configs

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


pd.set_option('display.float_format', lambda x: '%.2f' % x) #round float
plt.rcParams['figure.figsize'] = (8.5, 5)
plt.rcParams["patch.force_edgecolor"] = True
sns.mpl.rc("figure", figsize=(8.5,5))
pd.set_option('max_colwidth', 400)

# II. Import Data

We start by acquiring the training and testing datasets into Pandas DataFrames.

In [2]:
train_df = pd.read_csv('/work/train.csv', low_memory=False)
test_df = pd.read_csv('/work/test.csv', low_memory=False)

# III. Qu'est-ce qu'un pipeline et pourquoi l'utiliser ?

Le constructor Pipeline de scikit-learn permet de "chainer" une série de transformeurs et d'estimateurs.
Par exemple, si le modèle implique une sélection des features, une standardisation et ensuite une regression, ces trois étapes peuvent être encapsulées ensemble via une pipeline.

**Avantages** : 

* plus de lisibilité
* code réutilisable et facile à intégrer dans différents endroits du code
* plus simple d'expérimenter avec plusieurs modèles
* assure que chaque transformation de la données est exécutée dans le bonne ordre, et protège contre le data leakage pendant la cross-validation
* possibilité d'utiliser un grid search sur tous les parameters de tous les transformeurs et estimateurs.
* seulement besoin d'appeler la méthode fit et predict une fois sur la donnée pour fit l'entièrté des estimateurs.

**Construction d'une pipepline**

*Pipeline*

Pour construire une Pipeline, on utilise on liste de (key, value) où la key est une chaine de caractères contenant le nom donné à l'étape, et où la value est le nom de l'objet estimateur.
 
 ```
pipeline = Pipeline([
    ('model1', LinearRegression()),
    ('model2', SVC())
])

 ```

*make_pipeline*

 Il est aussi possible de construire une pipeline avec la fonction make_pipeline de scikit-learn. C'est en quelque sorte un raccourcis pour créer une pipeline qui peut prendre un nombre variable d'estimateurs et retourne une pipeline. Le nom donné à chaque étape est rempli automatiquement.

 ```
make_pipeline(Binazer(), MultinomialNB())
return => Pipeline(steps=[('Binarizer', Binarizer()), ('multinomialnb', MultinomialNB())])
 ```

# IV. Qu'est-ce qu'un transformer et pourquoi/comment l'utiliser

Un Transformer est un objet scikit-learn qui permet d'appliquer une transformation (encodage, normalisation, ...) sur un DataFrame. L'intérêt du Transformer est double.

    - Pouvoir définir une méthode de transformation sur-mesure.
    - Pouvoir appliquer cette méthode sur un DataFrame quelconque.

il existe plusieurs transformer: 
 
*exemple :* 

**OrdinalEncoder :**  convertit les variables catégorielles en variables numériques : 

*exemple:*

In [3]:
'''
enc = preprocessing.OrdinalEncoder()
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]

enc.fit(X)

enc.transform([['female', 'from US', 'uses Safari']])
'''

"\nenc = preprocessing.OrdinalEncoder()\nX = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]\n\nenc.fit(X)\n\nenc.transform([['female', 'from US', 'uses Safari']])\n"

**OneHotEncoder:** pour les variables catégorielles, crée de nouvelles variables indiquant la présence de chaque valeur possible à partir des données d'origine. 

*exemple:*
 

In [4]:
'''
one_hot_encoder = Pipeline(

  steps=[

    ('one_hot', OneHotEncoder(handle_unknown='ignore'))

  ]

)
'''

"\none_hot_encoder = Pipeline(\n\n  steps=[\n\n    ('one_hot', OneHotEncoder(handle_unknown='ignore'))\n\n  ]\n\n)\n"

il est également possible de construire un Transformer sur mesure avec FunctionTransformer. En particulier, pour certaines variables explicatives pour lesquelles il n'est pas nécessaire d'utiliser un OneHotEncode.

*exemple :* 
 

In [5]:
'''
encoding = { "low": 0, "med": 1, "high": 2, "vhigh": 3 }

def grad_encoder(df):

  for col in df.columns:

    df[col] = df[col].apply(lambda x: encoding[x])

  return df

eval_encoder = Pipeline(

  steps=[

    ('grad', FunctionTransformer(grad_encoder))

  ]

)
'''

'\nencoding = { "low": 0, "med": 1, "high": 2, "vhigh": 3 }\n\ndef grad_encoder(df):\n\n  for col in df.columns:\n\n    df[col] = df[col].apply(lambda x: encoding[x])\n\n  return df\n\neval_encoder = Pipeline(\n\n  steps=[\n\n    (\'grad\', FunctionTransformer(grad_encoder))\n\n  ]\n\n)\n'

ou pour des cas particuliers (exemple : si la valeur 5more est présente dans une colonne. Si seule la valeur numérique est intressante, il faudra construire un transformer sur mesure. C'est à dire une fonction qui sera appliquée à la colonne).

*exemple:*

In [6]:
'''
def num_encoder(df):

  for col in df.columns:

    df[col] = df[col].apply(lambda x: 5 if x == "5more" else x)

    df[col] = df[col].apply(lambda x: 6 if x == "more" else x)

  return df


num_encoder = Pipeline(

  steps=[

    ('num', FunctionTransformer(num_encoder))

  ]

)
'''

'\ndef num_encoder(df):\n\n  for col in df.columns:\n\n    df[col] = df[col].apply(lambda x: 5 if x == "5more" else x)\n\n    df[col] = df[col].apply(lambda x: 6 if x == "more" else x)\n\n  return df\n\n\nnum_encoder = Pipeline(\n\n  steps=[\n\n    (\'num\', FunctionTransformer(num_encoder))\n\n  ]\n\n)\n'

Une fois terminé, il ne restera plus qu'à combiner tous ces Transformers dans un ColumnTransformer : cela va permettre d'appliquer chaque Transformer sur un ensemble de colonnes sur-mesure. 
Plutôt que d'écrire un Transformer par colonne, cela permet de gagner du temps en appliquant la même méthode d'encodage sur plusieurs colonnes.

*exemple:*

In [7]:
'''
from sklearn.compose import ColumnTransformer


preprocessor = ColumnTransformer(

  transformers=[

    ('categorical', one_hot_encoder, ['lug_boot', 'safety']),

    ('grad', eval_encoder, ['buying', 'maint']),

    ('num', num_encoder, ['doors', 'persons']),

  ]

)
'''

"\nfrom sklearn.compose import ColumnTransformer\n\n\npreprocessor = ColumnTransformer(\n\n  transformers=[\n\n    ('categorical', one_hot_encoder, ['lug_boot', 'safety']),\n\n    ('grad', eval_encoder, ['buying', 'maint']),\n\n    ('num', num_encoder, ['doors', 'persons']),\n\n  ]\n\n)\n"

# V. Un estimator en machine learning

Dans Scikit-learn, un estimator est un objet (classifieur) qui implémente la méthode fit(X, y) pour estimer un modèle et predict(T) afin d’utiliser ce modèle pour prendre des décisions (faire des prédictions). IL possède une grille d'hyper-paramètres (parm_grid)et un itérateur de validations croisés (CV). Le paramètre parm_grid est un dictionnaire (ou une liste de dictionnaires) dont les clées sont les noms des hyper-paramètres à faire varier et les valeurs associées sont des listes de valeurs à tester.   

** Les différentes étapes dans l'utilisation d'un estimator: **
 * choisir les variables à tester et la variable cible (X et y)
 * l’ensemble de données peut être modifié de manière à ce qu’un pourcentage puisse être utilisé pour la formation du modèle et le reste pour les tests (train_test_split())
 * choisir un model (ex: regression logistique)
 * choisir les hyper-paramètres
 * fitter le modèle
 * appliquer le modèle sur les données de test.Dans le cas d'un apprentissage supervisé, utilisation de la fonction predict() pour avoir une prédiction. POur un apprentissage non supervisé, utilsiation de predict() ou transform pour déduire la variable cible
 

# VI. Application sur les données Titanic: 

**a ) Entrainement sur une partie des données et validation sur une autre **

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# variable cible
y = train_df["Survived"]

#variables explicatives 
features = ["Pclass", "Sex", "SibSp", "Parch"]
X = train_df[features]

# utilisation du tranformer OneHotEncoder
categorical_transformer = OneHotEncoder(handle_unknown="ignore")
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, features)
    ]
)

# modèle avec les hyper-paramètres (ici, randomforest: apprentissage supervisé)
model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=2)

#Pipeline 
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])


# train_test_split pour définir un pourcentage pour l'entrainement et le reste pour les test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)


# fitter le modèle
pipeline.fit(X_train, y_train)

# prédire 
predictions = pipeline.predict(X_test)


**a ) Entrainement l'ensemble des données **

In [9]:
from sklearn.ensemble import RandomForestClassifier

# variable cible
y = train_df["Survived"]

#variables explicatives 
features = ["Pclass", "Sex", "SibSp", "Parch"]
X = train_df[features]

# définition des variables utilisées pour la prediction
X_test = test_df[features]

# utilisation du tranformer OneHotEncoder
categorical_transformer = OneHotEncoder(handle_unknown="ignore")
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, features)
    ]
)
# modèle avec les hyper-paramètres (ici, randomforest: apprentissage supervisé)
model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=2)

#Pipeline 
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])



# fitter le modèle
pipeline.fit(X, y)

# prédire 
predictions = pipeline.predict(X_test)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=afb2cb51-127c-46d5-8892-aab192039534' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>